<a href="https://colab.research.google.com/github/AndriyDragan/HawkEye/blob/main/HawkEye.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TODO: Write project description and motivation

In [13]:
import os
import xml.etree.ElementTree as ET
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import random

data_dir = 'HawkEye/Data'

# Define function to read annotations
def read_annotations(xml_path):
    # Parse the XML file
    tree = ET.parse(xml_path)

    # Get the root element of the XML tree
    root = tree.getroot()

    # Initialize an empty list to store the annotations
    annotations = []

    # Loop through <object> elements in the XML
    for obj in root.findall('object'):

        # Extract the label from the <name> element
        name = obj.find('name').text

        # Extract the bounding box coordinates from the <bndbox> element
        bbox = obj.find('bndbox')
        xmin = int(bbox.find('xmin').text)
        ymin = int(bbox.find('ymin').text)
        xmax = int(bbox.find('xmax').text)
        ymax = int(bbox.find('ymax').text)
        annotations.append((name, (xmin, ymin, xmax, ymax)))

    # Return the list of annotations
    return annotations

# Define function to read images and annotations
def read_data(file_names, data_dir):
    # Initialize two empty lists
    data = []
    dims = []

    # Loop over the file names extracted and read the images and annotations
    for file_name in file_names:

        # Get the paths to the files
        img_path = os.path.join(data_dir, 'Images', file_name + '.jpg')
        xml_path = os.path.join(data_dir, 'Labels', 'Horizontal Bounding Boxes', file_name + '.xml')

        # Read the image
        img = Image.open(img_path)

        # Exclude 4-channel images
        if len(np.array(img).shape) != 3 or np.array(img).shape[2] != 3:
            continue

        # Extract annotations
        annotations = read_annotations(xml_path)

        # Get the dimensions of all objects and store it for later use
        for annot in annotations:
            width = abs(annot[1][0] - annot[1][2])
            height = abs(annot[1][1] - annot[1][3])
            dims.append(width)
            dims.append(height)

        data.append((img, annotations))

    return data, dims

# Function to display image with annotations
def display_image_with_annotations(image, annotations):
    # Create a copy of the image to draw on
    img_copy = image.copy()
    draw = ImageDraw.Draw(img_copy)

    # Draw annotations on the copy
    for obj in annotations:
        label, (xmin, ymin, xmax, ymax) = obj
        draw.rectangle([xmin, ymin, xmax, ymax], outline='red')
        draw.text((xmin, ymin), label, fill='red')

    # Convert to numpy array for displaying with matplotlib
    img_array = np.array(img_copy)
    plt.imshow(img_array)
    plt.axis('off')
    plt.show()


In [ ]:
# Get the list of filenames (without extension)
file_names = [f.split('.')[0] for f in os.listdir(os.path.join(data_dir, 'Images'))]

# Read the data
train_data, train_obj_dims = read_data(file_names, data_dir)

In [ ]:
# Display random samples
cnt = 1
plt.figure(figsize=(15, 10))
for r in range(3):
    for c in range(6):
        idx = random.randint(0, len(train_data) - 1)
        img = train_data[idx][0]
        annotations = train_data[idx][1]
        plt.subplot(3, 6, cnt)
        display_image_with_annotations(img, annotations)
        cnt += 1
plt.show()